In [ ]:
import stripe
import pandas as pd
from datetime import datetime
import numpy as np
# pd.reset_option('^display.', silent=True)  # Restore default display prefs

stripe.api_key = open("../creds/stripe_key", 'r').read()
stripe.api_version = "2020-08-27"

In [ ]:
def stripe_get_data(resource, start_date=None, end_date=None, **kwargs):   
    if start_date:
        # convert to unix timestamp
        start_date = int(start_date.timestamp())
    if end_date:
        # convert to unix timestamp
        end_date = int(end_date.timestamp())
    resource_list = getattr(stripe, resource).list(limit=100, created={"gte": start_date,"lt": end_date}, **kwargs)
    lst = []
    for i in resource_list.auto_paging_iter():
        lst.extend([i])
    df = pd.DataFrame(lst)
    if len(df) > 0:
        df['created'] = pd.to_datetime(df['created'], unit='s')
    return df

def contact_details(charges_field):
    billing_details = charges_field['data'][0].get('billing_details')
    contact = ', '.join([billing_details.name, billing_details.email])
    return contact

def generate_donation_report(junto_id, take_rate = .3):
    s = [f'Donations for {junto_id}']
    # Total per Junto
    filtered_df = donations[df['meta_juntoId']==junto_id].filter(['amount_received', 'created', 'contact'])
    total_donations = filtered_df['amount_received'].sum()
    payout = total_donations * (1-take_rate)
    s.append(f'Total Donations: {total_donations}')
    s.append(f'Total Payout: {payout}')
    s.append('\n')
    s.append(filtered_df.to_string(
        header=True,
        index=False,
        index_names=False
    ))
    return '\n'.join(s)

In [ ]:
df = stripe_get_data('PaymentIntent')
df = df.join(pd.json_normalize(df.metadata).add_prefix('meta_'))  # Flatten metadata fields

# pd.json_normalize(df['charges'])['data'][0][0].get('billing_details').keys().email

In [ ]:
donations = df[(df['meta_type']=='one_time_donation') & (df['status']=='succeeded')]
donations['amount_received'] = donations['amount_received'].div(100).round(2)
donations['contact'] = donations.apply(lambda row: contact_details(row['charges']), axis = 1)

In [ ]:
junto_id = '53FXvTKVnJlUPInVgDzd'
# junto_id = 'allsides-talks'
take_rate = .2
report = generate_donation_report(junto_id, take_rate)
with open(f'/Users/benjaminturtel/Downloads/donation_report_{junto_id}', 'w') as f:
    f.write(report)

print(report)